# Set-Up APIs, Dependencies, Function Definitions, etc.

In [1]:
import time
import numpy as np
import pandas as pd
from credentials import *
from datetime import datetime
import requests
import re
import tweepy
import spacy
import sqlalchemy as db
from config import *
import json
import pickle
from country_list import countries_for_language

nlp = spacy.load('en_core_web_sm')

In [2]:
def twitter_setup():
    """Set up Tweepy. 
    
    This function borrowed from Rodolfo Ferro, 
    https://dev.to/rodolfoferro/sentiment-analysis-on-trumpss-tweets-using-python-"""
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    
    api = tweepy.API(auth)
    return api

tweet_catcher = twitter_setup()

In [3]:
def get_candidate_tweets(handle, pagenum): #page 1 is first
    """
    Download tweets using Tweepy wrapper for Twitter API.
    
    Params:
    handle -- string, the target user's Twitter handle
    pagenum -- int, the cursor pagination
    returns: a list of Tweepy.Tweet objects"""
    tweetlist = tweet_catcher.user_timeline(screen_name=handle, count=200, page=str(pagenum), tweet_mode="extended")
    return tweetlist

def get_base_tweets(handlelist):
    """Download a sample set of tweets (approx. 600) for each handle in a list.
    
    Param:
    handlelist -- list of strings containing the handles of Twitter accounts to download.
    returns: a list of JSON-format tweet objects.
    """
    tweetlist = []
    for handle in handlelist:
        candidate_tweets = []
        page = 1
        while page < 2:
            tweets = get_candidate_tweets(handle, str(page))
            page += 1
            for tweet in tweets:
                candidate_tweets.append(tweet._json)
        for tweet in candidate_tweets:
            tweetlist.append(tweet)
    return tweetlist

def filter_quoted_tweet(tweet):
    """Return the original quoted tweet object.
    
    Param:
    tweet -- a JSON-format tweet object
    returns: None if not a quoted tweet, otherwise the original tweet object
    from which the passed tweet was quoted."""
    if "quoted_status" in tweet:
        return tweet["quoted_status"]
    else:
        return None

def filter_retweet(tweet):
    """Return the original of a retweeted tweet.
    
    Param:
    tweet -- a JSON-format tweet object
    returns: None if not a retweet, otherwise the original tweet object
    from which the passed tweet was retweeted."""
    if "retweeted_status" in tweet:
        return tweet["retweeted_status"]
    else:
        return None
    
#use tweet["full_text"]
def clean_tweet(tweet_text):
    """Prepare a tweet for natural language processing.
    
    Param:
    tweet -- the full text of a tweet
    returns: a string, the tweet's text after formatting.
    """
    tweet_text = re.sub(r'\S+\.\S+\/\S+', "", tweet_text) #remove links
    tweet_text = re.sub(r'@([A-Za-z0-9]+)', r'\1', tweet_text) #replace "@username" with "username"
    while "#" in tweet_text:
        tweet_text = re.sub(r'#(.+)', r'\1', tweet_text) #replace "#hashtag" with "hashtag"
    tweet_text = re.sub(r'\s(rt)|(RT)', '', tweet_text) #remove "RT" indicating retweet
    tweet_text = tweet_text.replace("\n", " ") # remove \n characters
    tweet_text = re.sub(r'[^\u0000-\u0080]', "", tweet_text) #remove non-ascii utf8 chars (ie, emojis)
    html_issues = {"&amp" : "and",
                   "&gt" : ">",
                   "&lt" : "<",
                  }
    for prob, sol in html_issues.items():
        tweet_text = tweet_text.replace(prob, sol)
    tweet_text = tweet_text.lower()
    return tweet_text

def process_texts(nlplist):
    """Use SpaCy API to remove stopwords, numerals, and whitespaces 
    from a list of tweet texts.
    
    Param:
    nlplist -- a list of SpaCy text objects.
    """
    processed_texts = []
    for tweet in nlplist:
        ptweet = remove_tokens(tweet)
        processed_texts.append(ptweet)
    return processed_texts

def remove_tokens(onenlptweet):
    """Helper function to process_texts(). Remove stopwords, spaces, and numerals.
    
    Param:
    onenlptweet -- a SpaCy text object tokenized and POS-tagged.
    """
    donetweet = []
    for token in onenlptweet:
        if not token.is_stop and not token.is_punct and not token.like_num and not token.is_space:
            donetweet.append(token.lemma_)
    return donetweet

In [4]:
import time
def get_nyt_hits(page, start_date, end_date):
    """Display the number of hits in a New York Times article search API call 
    for a given date range.
    
    Params:
    page -- int between 0 and 100, the response cursor's pagination.
    start_date -- string, begin date of the date range in YYYYMMDD format
    end_date -- string, end date of the date range in YYYYMMDD format
    """
    r1 = requests.get(f'https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=news_desk:("National" "Politics" "U.S.")&begin_date={start_date}&end_date={end_date}&page={page}&sort=oldest&api-key={NYT_KEY}')
    print("hits: ", r1.json()["response"]["meta"]["hits"])

def get_all_articles(page, start_date, end_date):
    """Get article data from New York Times article search API call for given date range.
    
    Params:
    page -- int between 0 and 100, response cursor's pagination.
    start_date -- string, begin date of the date range in YYYYMMDD format
    end_date -- string, end date of the date range in YYYYMMDD format
    returns: list of JSON objects containing article metadata including headlines and snippets.
    """
    r1 = requests.get(f'https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=news_desk:("National" "Politics" "U.S.")&begin_date={start_date}&end_date={end_date}&page={page}&sort=oldest&api-key={NYT_KEY}')
    articles = [obj for obj in r1.json()["response"]["docs"]]
    while page < 100:
        page += 1
        r2 = requests.get(f'https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=news_desk:("National" "Politics" "U.S.")&begin_date={start_date}&end_date={end_date}&page={page}&sort=oldest&api-key={NYT_KEY}')
        newarticles = [obj for obj in r2.json()["response"]["docs"]]
        for item in newarticles:
            articles.append(item)
        time.sleep(6)
    return articles

#(obj["headline"]["main"].lower(), obj["snippet"].lower(), obj["pub_date"])

In [6]:
engine = db.create_engine(f'mysql+mysqldb://{user}:{pw}@{host}/trailtracker?charset=utf8mb4', encoding='utf-8')

In [7]:
conn = engine.connect()
meta = db.MetaData()

In [108]:
conn.close()

# Collect Base Tweets

In [5]:
h1 = ["CoryBooker", "PeteButtigieg", "JulianCastro", "JohnDelaney", "TulsiGabbard"] 
h2 = ["SenGillibrand", "KamalaHarris", "Hickenlooper", "JayInslee", "amyklobuchar"] 
h3 = ["BetoORourke", "ewarren", "marwilliamson", "AndrewYang", "realDonaldTrump"]
h4 = ["SenSanders", "TimRyan", "MikeGravel", "ericswalwell", "WayneMessam"]
h5 = ["sethmoulton", "GovBillWeld", "JoeBiden", "BilldeBlasio", "GovernorBullock", "MichaelBennet"] 
allhandles = h1 + h2 + h3 + h4 + h5
candidate_fn = ["Cory", "Pete", "Julian", "John", "Tulsi", "Kirsten", "Kamala", "John", "Jay", "Amy", "Beto", "Elizabeth", "Marianne", "Andrew", "Donald", "Bernie", "Tim", "Mike", "Eric", "Wayne", "Seth", "Bill", "Joe", "Bill", "Steve", "Michael"]
candidate_ln = ["Booker", "Buttigieg", "Castro", "Delaney", "Gabbard", "Gillibrand", "Harris", "Hickenlooper", "Inslee", "Klobuchar", "O'Rourke", "Warren", "Williamson", "Yang", "Trump", "Sanders", "Ryan", "Gravel", "Swalwell", "Messam", "Moulton", "Weld", "Biden", "de Blasio", "Bullock", "Bennet"]
date_announced = ["2019-02-01", "2019-01-23", "2019-01-12", "2017-07-28", "2019-01-11", "2019-01-15", "2019-01-21", "2019-03-04", "2019-03-01", "2019-02-10", "2019-03-14", "2018-12-31", "2019-01-28", "2018-11-06", None, "2019-02-19", "2019-04-04", "2019-04-04", "2019-04-08", "2019-03-28", "2019-04-22", "2019-04-15", "2019-04-25", "2019-05-16", "2019-05-14", "2019-05-02"]
len(date_announced)

26

In [8]:
#200 most recent tweets per candidate
l1 = get_base_tweets(h1)

In [9]:
l2 = get_base_tweets(h2)

In [10]:
l3 = get_base_tweets(h3)

In [11]:
l4 = get_base_tweets(h4)

In [12]:
l5 = get_base_tweets(h5)

In [13]:
collected_tweets = l1 + l2 + l3 + l4 + l5

In [14]:
len(collected_tweets)

5200

##### Pickling tweet objects as needed

In [15]:
fileout = open("tweetsaver", "wb")
pickle.dump(collected_tweets, fileout)
fileout.close()

In [6]:
filein = open("tweetsaver", "rb")
collected_tweets = pickle.load(filein)
filein.close()

In [7]:
len(collected_tweets)

5200

### Filter out retweets and quoted tweets for database processing

In [8]:
rts = [filter_retweet(tweet) for tweet in collected_tweets]
qts = [filter_quoted_tweet(tweet) for tweet in collected_tweets]
qts2 = [tweet for tweet in qts if tweet != None]
rts2 = [tweet for tweet in rts if tweet != None]
qrts = rts2 + qts2
tweets_noqrts = [tweet for tweet in collected_tweets if "quoted_status" not in tweet and "retweeted_status" not in tweet]
tweets_combined = tweets_noqrts + qrts

# Collect NYT Headlines and Article Descriptions

In [18]:
get_nyt_hits(0, 20181106, 20190201)

hits:  871


In [19]:
articles1 = get_all_articles(0, 20181106, 20190201)

In [21]:
get_nyt_hits(0, 20190202, 20190501)

hits:  825


In [22]:
articles2 = get_all_articles(0, 20190202, 20190501)

In [23]:
allarticles = articles1 + articles2

In [24]:
nyt_fileout = open("articlesaver", "wb")
pickle.dump(allarticles, nyt_fileout)
nyt_fileout.close()

In [9]:
nyt_filein = open("articlesaver", "rb")
allarticles = pickle.load(nyt_filein)
nyt_filein.close()

# Text Processing

In [10]:
rt_texts = [clean_tweet(tweet["full_text"]) for tweet in rts if tweet]
qt_texts = [clean_tweet(tweet["full_text"]) for tweet in qts if tweet]
tweet_texts = [clean_tweet(tweet["full_text"]) for tweet in collected_tweets if "quoted_status" not in tweet and "retweeted_status" not in tweet]

In [11]:
nyt_headlines = [obj["headline"]["main"].lower() for obj in allarticles]

In [12]:
nyt_descriptions = [obj["snippet"].lower() for obj in allarticles]

In [13]:
all_texts = rt_texts + qt_texts + tweet_texts + nyt_headlines + nyt_descriptions
tweet_texts_combined = tweet_texts + rt_texts + qt_texts
len(all_texts)

8592

In [14]:
texts_nlp = [nlp(text) for text in all_texts]

In [15]:
processed_texts = process_texts(texts_nlp)

In [25]:
bigram = Phrases(processed_texts)
bigrammed_texts = [bigram[text] for text in processed_texts]

In [26]:
trigram = Phrases(bigrammed_texts)
trigrammed_texts = [trigram[text] for text in bigrammed_texts]

In [16]:
tweets_nlp = [nlp(tweet) for tweet in tweet_texts_combined]
articles_nlp = [nlp(headline) for headline in nyt_headlines]

# Topic Sorting

A naive keyword classification of tweets into topics.

In [19]:
# topics_keywords = {
#     "Immigration" : ["immigration", "immigrant", "border", "migrant", "asylum"],
#     "Taxes": ["tax", "taxation", " irs "],
#     "Economy": ["economy", "economic", "recession", "gdp", "income", "inflation", "monetary"],
#     "Racism": ["race", "racism", "racist", "anti-black", "white supremacy", "supremacist", "hate crime", "minority", "hatred", "reparations"],
#     "Foreign Policy": ["state department", "foreign policy", "china", "russia", "brazil", "india", "nato", "europe", "asia", "africa", "south africa", "south america", "mexico", "canada", "syria", "afghanistan", "iraq", "turkey", "israel", "egypt", "korea", "japan", "germany", "england", "eu", "uk", "france"],
#     "Poverty": ["socioeconomic", "class", "the rich", "poor", "poverty", "low-income", "working people"],
#     "Terrorism": ["terror", "terrorist", "terrorism", "isil", "isis", "al-qaeda", "daesh", "al-shabab", "islamic state", "bombing", "dhs"],
#     "Guns": ["gun", "nra", "bump stock", "shooting"],
#     "Education": ["education", "school", "curriculum"],
#     "Health Care": ["medicare for all", "health care", "healthcare", "obamacare", "trumpcare", "aca", "health exchange", "single-payer", "public option", "insurance", "uninsured", "preexisting condition"],
#     "Environment": ["environment", "green new deal", "climate change", "global warming", "carbon", "emissions", "pollution", "water supply", "environmental"],
#     "LGBTQ+": ["lgbt", "lgbtq", "gay", "lesbian", "bisexual", "transgender", "gender identity", "trans", "cis", "heterosexism"],
#     "Sexism": ["sexism", "patriarchy", "gender"],
#     "Social Security": ["social security", "retirement", "pension", "ira"],
#     "Employment": ["employment", "jobs", "working", "work", "unemployment", "unemployed"],
#     "Refugees": ["refugee", "displaced person", "displaced people"],
#     "Religion": ["religion", "faith", "god", "christianity", "judaism", "islam", "christian", "jew", "jewish", "muslim", "hinduism", "hindu", "buddhism", "buddhist", "atheist", "atheism", "humanism", "humanist", "secular"],
#     "Drugs": ["drug", "marijuana", "opiod", "heroin", "cocaine", " dea "],
#     "Policing": ["police", "prison", "law enforcement", "policing"]
# }

countries = [re.compile(f'{country[1]}'.lower()) for country in countries_for_language('en')]
fp_words = [re.compile(r'state department'), re.compile(r'foreign policy'), re.compile(r'\bnato\b'), re.compile(r'european union'), re.compile(r'\beu\b'), re.compile(r'african union'), re.compile(r'\bau\b'), re.compile(r'african?'), re.compile(r'pentagon'), re.compile(r'\bwar\b')]

for word in fp_words:
    countries.append(word)

topics_keywords = {
    "Immigration" : [re.compile(r'immigr.*'), re.compile(r'borders?'), re.compile(r'migrants?'), re.compile(r'asylum')],
    "Taxes": [re.compile(r'\btax(es)?\b'), re.compile(r'\birs\b')],
    "Economy": [re.compile(r'\beconom.*\b'), re.compile(r'recessions?'), re.compile(r'\bgdp\b'), re.compile(r'incomes?'), re.compile(r'inflation'), re.compile(r'monetary')],
    "Racism": [re.compile(r'\brace\b'), re.compile(r'racis[t|m]s?'), re.compile(r'anti-black'), re.compile(r'white suprem(acy|acist)s?'), re.compile(r'hate crimes?'), re.compile(r'minorit(ies|y)'), re.compile(r'\breparations\b')],
    "Foreign Policy": countries,
    "Poverty": [re.compile(r'socioeconomic'), re.compile(r'the rich'), re.compile(r'poor'), re.compile(r'poverty'), re.compile(r'low-?\s?income'), re.compile(r'working-?\s?(people|class)')],
    "Terrorism": [re.compile(r'terror(ist|ism)?s?'), re.compile(r'\bisi(l|s)\b'), re.compile(r'al-qaeda'), re.compile(r'daesh'), re.compile(r'al-shabab'), re.compile(r'islamic\sstate'), re.compile(r'bomb(ing)?s?'), re.compile(r'\bdhs\b'), re.compile(r'homeland\ssecurity')],
    "Guns": [re.compile(r'guns?'), re.compile(r'\bnra\b'), re.compile(r'bump\sstock'), re.compile(r'shoo?t(ing)?s?')],
    "Education": [re.compile(r'education'), re.compile(r'schools?'), re.compile(r'curricul(um|a)?s?')],
    "Health Care": [re.compile(r'medicare'), re.compile(r'medicaid'), re.compile(r'(health|obama|trump)-?\s?care'), re.compile(r'\baca\b'), re.compile(r'affordable\scare\sact'), re.compile(r'single-?\s?payer'), re.compile(r'public\soption'), re.compile(r'(un)?insur(ed|ance)'), "uninsured", "preexisting condition"],
    "Environment": [re.compile(r'environment(al)?(ism)?'), re.compile(r'green\snew\sdeal'), re.compile(r'climate\schange'), re.compile(r'global\swarming'), re.compile(r'\bcarbon\b'), re.compile(r'\bemissions?\b'), re.compile(r'pollution'), re.compile(r'water\ssupply')],
    "LGBTQ+": [re.compile(r'lgbtq?'), re.compile(r'gay'), re.compile(r'lesbian'), re.compile(r'\bbi(sexual)?\b'), re.compile(r'\btrans(gender)?\b'), re.compile(r'gender\sidentity'), re.compile(r'\btrans(gender)?\b'), re.compile(r'\bcis(gender)?\b'), re.compile(r'heterosex')],
    "Sexism": [re.compile(r'\bsexism\b'), re.compile(r'(pay)|(wage)\sgap'), re.compile(r'patriarch')],
    "Social Security": [re.compile(r'social\ssecurity'), re.compile(r'\bretired?(ment)?\b'), re.compile(r'\bpension(er)?\b'), re.compile(r'\bira\b')],
    "Employment": [re.compile(r'\b(un)?(under)?-?employ(ed)?(ment)?(er)?s?\b'), re.compile(r'\bjobs?\b'), re.compile(r'\bworks?(ing)?(ed)?\b')],
    "Refugees": [re.compile(r'refugee'), re.compile(r'displaced\s(person)|(people)s?')],
    "Religion": [re.compile(r'religio'), re.compile(r'faith'), re.compile(r'god'), re.compile(r'christian'), re.compile(r'judaism'), re.compile(r'islam'), re.compile(r'jew'), re.compile(r'muslim'), re.compile(r'hindu'), re.compile(r'buddha?'), re.compile(r'atheis'), re.compile(r'humanis'), re.compile(r'secular')],
    "Drugs": ["drug", "marijuana", "opiod", "heroin", "cocaine", " dea "],
    "Policing": ["police", "prison", "law enforcement", "policing"],
}

In [20]:
def tweet_sort(tweetlist, topic_hash, text_hash):
    """Classify tweets by topic keywords.
    
    Params:
    tweetlist -- list of JSON-format tweet objects
    topic_hash -- dictionary with keys as topics, values as lists of keywords
        (regex for better performance than totally-naive keywords)
    text_hash -- dictionary with keys as topics matching those of topic_hash,
        values as empty lists.
    returns: dictionary with keys as topics, values as lists of strings containing texts.
    """
    for tweet in tweetlist:
        for kvpair in topic_hash.items():
            for keyword in kvpair[1]:
                if keyword in clean_tweet(tweet["full_text"]):
                    text_hash[kvpair[0]].append(tweet)
    return text_hash

def article_sort(text_list, topic_hash, text_hash):
    """
    Classify headline texts by topic keywords.
    
    Params:
    text_list -- list of text of article headlines and/or snippets.
    topic_hash -- dictionary with keys as topics, values as lists of keywords
        (regex for better performance than totally-naive keywords)
    text_hash -- dictionary with keys as topics matching those of topic_hash,
        values as empty lists.
    returns: dictionary with keys as topics, values as lists of strings 
        (the sorted article texts).
    """
    for doc in text_list:
        for kvpair in topic_hash.items():
            for keyword in kvpair[1]:
                if keyword in doc:
                    text_hash[kvpair[0]].append(doc)
    return text_hash

In [21]:
empty_tweet_topics = {topic: [] for topic in topics_keywords.keys()}
tweets_by_topic = tweet_sort(tweets_combined, topics_keywords, empty_tweet_topics)

In [22]:
empty_article_topics = {topic: [] for topic in topics_keywords.keys()}
articles_by_topic = article_sort(nyt_headlines, topics_keywords, empty_article_topics)

# Format Data for insert into MySQL Database

### Topic_T

In [23]:
topics_df = pd.DataFrame({
    "TopicNum": [i for i in range(19)],
    "TopicName": [kw for kw in topics_keywords.keys()]
})

In [24]:
topics_df.head()

,TopicNum,TopicName
0,0,Immigration
1,1,Taxes
2,2,Economy
3,3,Racism
4,4,Foreign Policy


In [25]:
topics_df.to_sql("topic_t", engine, if_exists="append", index=False, method="multi")

In [36]:
conn.execute("ALTER TABLE topic_t ADD CONSTRAINT Topic_PK PRIMARY KEY(TopicNum)")

### Twitter_User_T

In [52]:
allusers = [tweet["user"]["name"] for tweet in tweets_combined]
unique_users = set(allusers)
unique_users_list = list(unique_users)
users_df = pd.DataFrame({
    "UserID": [i for i, t in enumerate(unique_users_list)],
    "UserName": [name for name in unique_users_list]
})
userdf2 = pd.DataFrame({
    "UserName": [tweet["user"]["name"] for tweet in tweets_combined]
})
userdf3 = userdf2.merge(users_df, how="left", on="UserName")
twitter_users_df = pd.DataFrame({
    "UserID": userdf3.UserID,
    "UserName": [tweet["user"]["name"] for tweet in tweets_combined],
    "TwitterHandle": [tweet["user"]["screen_name"] for tweet in tweets_combined]
})
twitter_users_df = twitter_users_df.drop_duplicates(subset="UserID")

In [54]:
twitter_users_df.head()

,UserID,UserName,TwitterHandle
0,832,Cory Booker,CoryBooker
102,219,Pete Buttigieg,PeteButtigieg
202,904,Julián Castro,JulianCastro
289,699,John Delaney,JohnDelaney
423,306,Tulsi Gabbard,TulsiGabbard


In [55]:
twitter_users_df.to_sql("twitter_user_t", engine, if_exists="append", index=False, method="multi")

In [ ]:
conn.execute("ALTER TABLE twitter_user_t ADD CONSTRAINT Twitter_User_PK PRIMARY KEY(UserID)")

In [45]:
twitter_user_t = db.Table("twitter_user_t", meta, autoload=True, autoload_with=engine)
s = db.select([twitter_user_t])
result = conn.execute(s)
for row in result:
    print(row)

### Candidate_T

In [56]:
cand_df = pd.DataFrame({
    "TwitterHandle": [handle for handle in allhandles]
})
cdf2 = cand_df.merge(twitter_users_df, how="left", on="TwitterHandle")
cdf2 = cdf2.drop_duplicates()
candidates_df = pd.DataFrame({
    "CandidateID": cdf2.UserID,
    "CandidateLastName": candidate_ln,
    "CandidateFirstName": candidate_fn,
    "DateAnnounced": date_announced,
    "Party": ["Democratic" if name != "Trump" and name != "Weld" else "Republican" for name in candidate_ln]
})

In [57]:
candidates_df.head()

,CandidateID,CandidateLastName,CandidateFirstName,DateAnnounced,Party
0,832,Booker,Cory,2019-02-01,Democratic
1,219,Buttigieg,Pete,2019-01-23,Democratic
2,904,Castro,Julian,2019-01-12,Democratic
3,699,Delaney,John,2017-07-28,Democratic
4,306,Gabbard,Tulsi,2019-01-11,Democratic


In [58]:
candidates_df.to_sql("candidate_t", engine, if_exists="append", index=False, method="multi")

In [ ]:
conn.execute("ALTER TABLE candidate_t ADD CONSTRAINT Candidate_PK PRIMARY KEY(CandidateID)")
conn.execute("ALTER TABLE candidate_t ADD CONSTRAINT Candidate_FK FOREIGN KEY (CandidateID) REFERENCES Twitter_User_T(UserID)")

### Tweet_T

In [62]:
tweets_df = pd.DataFrame({
    #"TweetID" : [i for i, tweet in enumerate(tweets_combined)],
    "TweetID" : [tweet["id_str"] for tweet in tweets_combined],
    "LengthWords": [len(tweet) for tweet in tweets_nlp],
    "TweetFullText": [tweet["full_text"] for tweet in tweets_combined],
    "TweetDate" : [time.strftime("%Y-%m-%d", time.strptime(tweet["created_at"],'%a %b %d %H:%M:%S +0000 %Y')) for tweet in tweets_combined],
    "TimesRetweeted" : [tweet["retweet_count"] for tweet in tweets_combined],
    "UserID" : userdf3.UserID
})
tweets_df = tweets_df.drop_duplicates(subset="TweetID")

In [63]:
tweets_df.head()

,TweetID,LengthWords,TweetFullText,TweetDate,TimesRetweeted,UserID
0,1132456883116285953,17,@RUSSBARNES Clearly I crack myself up too. Min...,2019-05-26,1,832
1,1132414330237014016,34,On the road in Iowa and the drives get long. S...,2019-05-25,507,832
2,1132332387763937280,40,The moment Joan told me she was going to caucu...,2019-05-25,79,832
3,1132280827713343488,19,@JennaSumar Are we even a family if we don't t...,2019-05-25,2,832
4,1132045252620169216,54,"The abortion bans in Georgia, Alabama, Missour...",2019-05-24,152,832


In [64]:
tweets_df.to_sql("tweet_t", engine, if_exists="append", index=False, method="multi")

In [ ]:
conn.execute("ALTER TABLE tweet_t ADD CONSTRAINT Tweet_PK PRIMARY KEY(TweetID)")
conn.execute("ALTER TABLE tweet_t ADD CONSTRAINT Tweet_FK_1 FOREIGN KEY(TweetDominantTopic) REFERENCES Topic_T(TopicNum)")
conn.execute("ALTER TABLE tweet_t ADD CONSTRAINT Tweet_FK_2 FOREIGN KEY(UserID) REFERENCES Twitter_User_T(UserID)")

### Retweet_T

In [68]:
tweetsdf2 = tweets_df.drop_duplicates(subset="TweetFullText")
rtdf1 = pd.DataFrame({
    "TweetFullText": [tweet["full_text"] for tweet in qrts]
})
rtdf2 = rtdf1.merge(tweetsdf2, how="left", on="TweetFullText")
retweets_df = pd.DataFrame({
    "UserID": rtdf2.UserID,
    "TweetID": rtdf2.TweetID,
    "OriginalTweetDate": [time.strftime("%Y-%m-%d", time.strptime(tweet["created_at"],'%a %b %d %H:%M:%S +0000 %Y')) for tweet in qrts]
})
retweets_df = retweets_df.drop_duplicates()

In [69]:
retweets_df.head()

,UserID,TweetID,OriginalTweetDate
0,992,1132049485088088065,2019-05-24
1,107,1131305477449629697,2019-05-22
2,565,1131942587240206336,2019-05-24
3,749,1129442151186784256,2019-05-17
4,53,1130890140119969793,2019-05-21


In [70]:
retweets_df.to_sql("retweet_t", engine, if_exists="append", index=False, method="multi")

In [ ]:
conn.execute("ALTER TABLE retweet_t ADD CONSTRAINT Retweet_PK PRIMARY KEY(UserID, TweetID, OriginalTweetDate)")
conn.execute("ALTER TABLE retweet_t ADD CONSTRAINT Retweet_FK_1 FOREIGN KEY(UserID) REFERENCES Twitter_User_T(UserID)")
conn.execute("ALTER TABLE retweet_t ADD CONSTRAINT Retweet_FK_2 FOREIGN KEY(TweetID) REFERENCES Tweet_T(TweetID)")

### Keyword_T

In [71]:
keywords_all_topics = [(kv[0], keyword) for kv in topics_keywords.items() for keyword in kv[1]]
keywords_df = pd.DataFrame({
    "TopicNum": [l[0] for t in keywords_all_topics for l in topics_df.values if t[0] == l[1]],
    "Keyword": [t[1] for t in keywords_all_topics]
})

In [72]:
keywords_df.head()

,TopicNum,Keyword
0,0,immigration
1,0,immigrant
2,0,border
3,0,migrant
4,0,asylum


In [73]:
keywords_df.to_sql("keyword_t", engine, if_exists="append", index=False, method="multi")

In [ ]:
conn.execute("ALTER TABLE keyword_t ADD CONSTRAINT Keyword_PK PRIMARY KEY(TopicNum, Keyword)")
conn.execute("ALTER TABLE keyword_t ADD CONSTRAINT Keyword_FK FOREIGN KEY (TopicNum) REFERENCES Topic_T(TopicNum)")

### Headline_T

In [74]:
headline_df = pd.DataFrame({
    "ArticleID": [i for i, article in enumerate(allarticles)],
    "ArticleHeadline": nyt_headlines,
    "ArticleDescription": nyt_descriptions,
    "DatePublished": [datetime.strptime(obj["pub_date"], "%Y-%m-%dT%H:%M:%S%z").strftime("%Y-%m-%d") for obj in allarticles],
})

In [75]:
headline_df.head()

,ArticleID,ArticleHeadline,ArticleDescription,DatePublished
0,0,georgia governor’s race is hurtling toward ele...,brian kemp and stacey abrams are both trying t...,2018-11-06
1,1,sign up for live updates from our reporters,you'll know as soon as we do the direction the...,2018-11-06
2,2,your top 12 questions about the midterms,our political reporter alex burns answers read...,2018-11-06
3,3,bad weather will greet voters from florida to ...,"rain on election day can decrease turnout, whi...",2018-11-06
4,4,on politics: election day is here,voters across the united states will choose ca...,2018-11-06


In [76]:
headline_df.to_sql("headline_t", engine, if_exists="append", index=False, method="multi")

In [ ]:
conn.execute("ALTER TABLE headline_t ADD CONSTRAINT Headline_PK PRIMARY KEY(ArticleID)")
conn.execute("ALTER TABLE headline_t ADD CONSTRAINT Headline_FK_1 FOREIGN KEY(NewsID) REFERENCES NewsSource_T(NewsID)")
conn.execute("ALTER TABLE headline_t ADD CONSTRAINT Headline_FK_2 FOREIGN KEY(HeadlineDominantTopic) REFERENCES Topic_T(TopicNum)")

### NGram_Token_T

In [77]:
allnlp = tweets_nlp+articles_nlp
alltokens = [token for doc in allnlp for token in doc]
alltokens_textpos = [(token.text, token.pos_) for doc in allnlp for token in doc]

In [78]:
unique_tokens_text = set(alltokens_textpos)
unique_tokens_list = list(unique_tokens_text)
unique_tokens = []
for token in alltokens:
    if (token.text, token.pos_) in unique_tokens_list:
        unique_tokens.append(token)
        unique_tokens_list.remove((token.text, token.pos_))
print(len(unique_tokens))

16645


In [79]:
ngramtoken_df = pd.DataFrame({
    "TokenID": [i for i, token in enumerate(unique_tokens)],
    "TokenText": [token.text for token in unique_tokens],
    "PartOfSpeech": [token.pos_ for token in unique_tokens],
    "TokenLemma": [token.lemma_ for token in unique_tokens]
})

In [80]:
ngramtoken_df.head()

,TokenID,TokenText,PartOfSpeech,TokenLemma
0,0,russbarnes,VERB,russbarn
1,1,clearly,ADV,clearly
2,2,i,PRON,i
3,3,crack,VERB,crack
4,4,myself,PRON,-PRON-


In [81]:
ngramtoken_df.to_sql("ngramtoken_t", engine, if_exists="append", index=False, method="multi")

In [ ]:
conn.execute("ALTER TABLE ngramtoken_t ADD CONSTRAINT NGramToken_PK PRIMARY KEY(TokenID)")

### TokenInTweet_T

In [111]:
tweet_and_nlp = list(zip(tweets_combined, tweets_nlp))

In [112]:
tokenintweet = [(token.text, token.pos_, item[0]["id_str"]) for item in tweet_and_nlp for token in item[1] if not token.is_punct and not token.is_space]

In [121]:
tokentweetseries = pd.DataFrame({
    "TokenText": [tup[0] for tup in tokenintweet],
    "PartOfSpeech": [tup[1] for tup in tokenintweet],
    "TweetID": [tup[2] for tup in tokenintweet]
})
tokentweetdf1 = tokentweetseries.merge(ngramtoken_df, how='left', on=['TokenText', 'PartOfSpeech'])
tokenintweet_df = pd.DataFrame({
    "TweetID": tokentweetdf1.TweetID,
    "TokenID": tokentweetdf1.TokenID
})
tokenintweet_df = tokenintweet_df.drop_duplicates()

In [122]:
tokenintweet_df.head()

,TweetID,TokenID
0,1132456883116285953,0
1,1132456883116285953,1
2,1132456883116285953,2
3,1132456883116285953,3
4,1132456883116285953,4


In [123]:
tokenintweet_df.to_sql("tokenintweet_t", engine, if_exists="append", index=False, chunksize=200, method="multi")

In [ ]:
conn.execute("ALTER TABLE tokenintweet_t ADD CONSTRAINT TokenInTweet_PK PRIMARY KEY(TokenID, TweetID)")
conn.execute("ALTER TABLE tokenintweet_t ADD CONSTRAINT TokenInTweet_FK_1 FOREIGN KEY(TokenID) REFERENCES NGramToken_T(TokenID)")
conn.execute("ALTER TABLE tokenintweet_t ADD CONSTRAINT TokenInTweet_FK_2 FOREIGN KEY(TweetID) REFERENCES Tweet_T(TweetID)")

### Hashtag_T

In [116]:
allhashtags = [tweet["entities"]["hashtags"] for tweet in tweets_combined if tweet["entities"]["hashtags"] != []]
tagtexts = [d["text"].lower() for l in allhashtags for d in l]
tagtexts_unique = list(set(tagtexts))
print("all tags len: ", len(tagtexts))
print("unique tags len: ", len(tagtexts_unique))

all tags len:  1844
unique tags len:  698


In [117]:
hashtag_df = pd.DataFrame({
    "HashtagID": [i for i, tag in enumerate(tagtexts_unique)],
    "HashtagName": tagtexts_unique
})

In [118]:
hashtag_df.head()

,HashtagID,HashtagName
0,0,moleg
1,1,florida
2,2,gobows
3,3,americanfamilyact
4,4,repealhyde


In [119]:
hashtag_df.to_sql("hashtag_t", engine, if_exists="append", index=False, method="multi")

In [ ]:
#conn.execute("ALTER TABLE hashtag_t ADD CONSTRAINT Hashtag_PK PRIMARY KEY(HashtagID)")

### Link_T

In [124]:
all_links = [tweet["entities"]["urls"] for tweet in tweets_combined if tweet["entities"]["urls"] != []]
links = [d["display_url"] for l in all_links for d in l]
link_df = pd.DataFrame({
    "LinkID": [i for i, l in enumerate(links)],
    "LinkURL": links
})

In [125]:
link_df.head()

,LinkID,LinkURL
0,0,coryb.kr/2X0FVfj
1,1,coryb.kr/2JxzuNJ
2,2,mobilize.us/bookerforiowa/…
3,3,nytimes.com/2019/05/23/opi…
4,4,coryb.kr/2JxzuNJ


In [126]:
link_df.to_sql("link_t", engine, if_exists="append", index=False, method="multi")

### UserMention_T

In [127]:
mentiontweets = [tweet for tweet in tweets_combined if tweet["entities"]["user_mentions"] != []]
usermentions = [tweet["entities"]["user_mentions"] for tweet in tweets_combined if tweet["entities"]["user_mentions"] != []]
tweets_mentioning_candidates = []
for tweet in tweets_combined:
    if tweet["entities"]["user_mentions"] != []:
        for d in tweet["entities"]["user_mentions"]:
                if d["screen_name"] in allhandles:
                    tweets_mentioning_candidates.append(tweet)
which_candidate_mentioned = [d["screen_name"] for l in usermentions for d in l if d["screen_name"] in allhandles]
mentionids = [(i, tweet) for i, tweet in enumerate(tweets_combined) if tweet in tweets_mentioning_candidates]

In [131]:
umdf0 = pd.DataFrame({
    "TweetFullText": [tweet["full_text"] for tweet in tweets_mentioning_candidates]
})
umdf2 = umdf0.merge(tweetsdf2, how="left", on="TweetFullText")
usermentiondf1 = pd.DataFrame({
    "TweetFullText": [tweet["full_text"] for tweet in tweets_mentioning_candidates],
    "UID": umdf2.UserID,
    "TwitterHandle": which_candidate_mentioned
})
usermdf15 = usermentiondf1.merge(cdf2, how="left", on="TwitterHandle")
usermdf2 = usermdf15.merge(tweetsdf2, how="left", on="TweetFullText")
usermention_df = pd.DataFrame({
    "UserID": usermdf2.UserID_y,
    "CandidateID": usermdf2.UserID_x,
    "TweetID": usermdf2.TweetID
})
usermention_df = usermention_df.drop_duplicates()

In [132]:
usermention_df.head()

,UserID,CandidateID,TweetID
0,699,425,1132036545672884224
1,306,115,1122285523786260480
2,306,404,1122285523786260480
3,306,690,1122285523786260480
4,306,425,1115239116948377600


In [133]:
usermention_df.to_sql("user_mention_t", engine, if_exists="append", index=False, method="multi")

### ArticleAboutCandidate_T

In [134]:
def replace_candidate_names(string_contains_name):
    for name in candidate_ln:
        if name in string_contains_name:
            return name

candidate_names = ["Booker, Cory", "Buttigieg, P", "Castro, J", "Delaney, J", "Gabbard, T", "Gillibrand", "Harris, Kamala", "Hickenlooper, J", "Inslee", "Klobuchar", "O'Rourke, Beto", "Warren, Elizabeth", "Williamson, Marianne", "Yang, Andrew", "Trump, Donald", "Sanders, B", "Ryan, Tim", "Gravel, M", "Swalwell, E", "Messam, Wayne", "Moulton, Seth", "Weld, William", "Biden, J", "de Blasio, B", "Bennet, Michael", "Bullock, Steve"]
articles_about_candidates = []
for article in allarticles:
    for d in article["keywords"]:
        for name in candidate_names:
            if name in d["value"]:
                articles_about_candidates.append(article["headline"]["main"].lower())

candidate_in_article = []
for article in allarticles:
    for d in article["keywords"]:
        for name in candidate_names:
            if name in d["value"]:
                candidate_in_article.append(d["value"])

candidate_articles_df1 = pd.DataFrame({
    "ArticleHeadline": articles_about_candidates,
})
cadf2 = candidate_articles_df1.merge(headline_df, how="left", on="ArticleHeadline")
cadf3 = pd.DataFrame({
    "ArticleID": cadf2.ArticleID,
    "CandidateLastName": candidate_in_article,
})
namelist = [replace_candidate_names(s) for s in cadf3.CandidateLastName]
cadf3["CandidateLastName"] = namelist
cadf4 = cadf3.merge(candidates_df, how="left", on="CandidateLastName")
article_about_candidate_df = pd.DataFrame({
    "ArticleID": cadf2.ArticleID,
    "CandidateID": cadf4.CandidateID
})


In [135]:
article_about_candidate_df.head()

,ArticleID,CandidateID
0,6,425
1,8,425
2,9,635
3,9,425
4,70,425


In [136]:
article_about_candidate_df.to_sql("articleaboutcandidate_t", engine, if_exists="append", index=False, method="multi")

### HashtagInTweet_T

In [140]:
text_ht = []
for tweet in tweets_combined:
    if tweet["entities"]["hashtags"] != []:
        for d in tweet["entities"]["hashtags"]:
                text_ht.append((d["text"].lower(), tweet["full_text"]))
htdf1 = pd.DataFrame({
    "TweetFullText": [tup[1] for tup in text_ht],
    "HashtagName": [tup[0] for tup in text_ht]
})
htdf2 = htdf1.merge(hashtag_df, how="left", on="HashtagName")
htdf3 = htdf2.merge(tweetsdf2, how="left", on="TweetFullText")
hashtag_in_tweet_df = pd.DataFrame({
    "HashtagID": htdf3.HashtagID,
    "TweetID": htdf3.TweetID
})
hashtag_in_tweet_df = hashtag_in_tweet_df.drop_duplicates()

In [141]:
hashtag_in_tweet_df.head()

,HashtagID,TweetID
0,485,1131569358587727872
1,325,1131318535811162112
2,398,1130922527285567488
3,681,1127622062309355521
4,641,1126479138838257665


In [142]:
hashtag_in_tweet_df.to_sql("hashtagintweet_t", engine, if_exists="append", index=False, method="multi")

### TokenInHeadline_T

In [146]:
tokeninheadline = [(i, token.text) for i, article in enumerate(articles_nlp) for token in article]
tokenhead_df1 = pd.DataFrame({
    "ArticleID": [i for i, t in tokeninheadline],
    "TokenText": [t for i, t in tokeninheadline]
})
tokenhead_df2 = tokenhead_df1.merge(ngramtoken_df, how="left", on="TokenText")
token_in_headline_df = pd.DataFrame({
    "TokenID": tokenhead_df2.TokenID,
    "ArticleID": tokenhead_df2.ArticleID
})
token_in_headline_df = token_in_headline_df.drop_duplicates()

In [147]:
token_in_headline_df.head()

,TokenID,ArticleID
0,86,0
1,4324,0
2,4485,0
3,14860,0
4,14926,0


In [148]:
token_in_headline_df.to_sql("tokeninheadline_t", engine, if_exists="append", index=False, method="multi")

### LinkInstanceTweet_T

In [149]:
tweets_with_links = []
for tweet in tweets_combined:
    if tweet["entities"]["urls"] != []:
        for d in tweet["entities"]["urls"]:
            tweets_with_links.append(tweet)
linktweetdf1 = pd.DataFrame({
    "LinkID": link_df.LinkID,
    "TweetFullText": [tweet["full_text"] for tweet in tweets_with_links]
})
linktweetdf2 = linktweetdf1.merge(tweetsdf2, how="left", on="TweetFullText")
linkinstancetweet_df = pd.DataFrame({
    "LinkID": link_df.LinkID,
    "TweetID": linktweetdf2.TweetID
})

In [150]:
linkinstancetweet_df.head()

,LinkID,TweetID
0,0,1132332387763937280
1,1,1132045252620169216
2,2,1132020165766451205
3,3,1131617526490783744
4,4,1131579847023779840


In [151]:
linkinstancetweet_df.to_sql("linkinstancetweet_t", engine, if_exists="append", index=False, method="multi")

### TopicInTweet_T

In [152]:
#tweets_by_topic
tweet_topic_pairs = [(kv[0], tweet["full_text"]) for kv in tweets_by_topic.items() for tweet in kv[1]]
tt_df1 = pd.DataFrame({
    "TopicNum": [l[0] for t in tweet_topic_pairs for l in topics_df.values if l[1] == t[0]],
    "TweetFullText": [t[1] for t in tweet_topic_pairs]
})

tt_df2 = tt_df1.merge(tweetsdf2, how="left", on="TweetFullText")
topicintweet_df = pd.DataFrame({
    "TopicNum": tt_df2.TopicNum,
    "TweetID": tt_df2.TweetID
})
topicintweet_df = topicintweet_df.drop_duplicates()

In [153]:
topicintweet_df.head()

,TopicNum,TweetID
0,0,1129127173598711809
2,0,1127258994513522688
3,0,1127257199892471810
4,0,1125938417341280256
6,0,1123756751306809347


In [154]:
topicintweet_df.to_sql("topicintweet_t", engine, if_exists="append", index=False, method="multi")

### TopicInHeadline_T

In [155]:
article_topic_pairs = [(kv[0], headline) for kv in articles_by_topic.items() for headline in kv[1]]
htopic_df1 = pd.DataFrame({
    "TopicNum": [l[0] for t in article_topic_pairs for l in topics_df.values if l[1] == t[0]],
    "ArticleHeadline": [t[1] for t in article_topic_pairs]
})
htopic_df2 = htopic_df1.merge(headline_df, how="left", on="ArticleHeadline")
topicinheadline_df = pd.DataFrame({
    "TopicNum": htopic_df2.TopicNum,
    "ArticleID": htopic_df2.ArticleID
})
topicinheadline_df = topicinheadline_df.drop_duplicates()

In [156]:
topicinheadline_df.head()

,TopicNum,ArticleID
0,0,137
1,0,162
2,0,267
3,0,268
5,0,277


In [157]:
topicinheadline_df.to_sql("topicinheadline_t", engine, if_exists="append", index=False, method="multi")

In [158]:
conn.close()